In [1]:
import pandas as pd
import numpy as np

## Load

In [2]:
def load_data(file_name):
    default_dir = "ch6/nobel_winners/data/"
    df = pd.read_json(open(default_dir + file_name))
    return df

In [ ]:
df = load_data('nobel_winners_full.json')

## Basic exploration

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include=['object'])

In [ ]:
df.head(5)

In [ ]:
set(df.born_in.apply(type))

In [ ]:
df[df.name.str.contains('\*')]['name']

## Remove * from names

In [ ]:
df.name = df.name.str.replace('*', '')
df.name = df.name.str.strip()

In [ ]:
df[df.name.str.contains('\*')]['name']

## Replace "" with np.nan

In [ ]:
df.replace("", np.nan, inplace=True)

## Save "born in" information to another table

In [ ]:
df_born = df[df.born_in.notnull()][["name", "year", "born_in", "text"]]
df_born.head()

Some information of the "born_in" column is wrong. For example, Aaron Klug was born in Litthuania actually. Howerver, I'll ignore them.

In [ ]:
df_born[df_born.duplicated(subset=["name", "year"], keep=False)].sort_values("name")

## Check duplicated rows

In [ ]:
df[df.duplicated(subset=['name', 'year'], keep=False)].sort_values('name')[['name', 'year', 'country', 'born_in', 'text']]

## Reindex and drop duplicates

In [ ]:
df = df.reindex(np.random.permutation(df.index)).sort_values("born_in")
df = df.drop_duplicates(['name', 'year'], keep='last')
df = df.sort_index()
df.count()

In [ ]:
df[df.born_in.notnull()]

In [ ]:
df[df.duplicated(subset=['name'], keep=False)].sort_values('name')[['name', 'year', 'country']]

## Fix errors manually

In [ ]:
df = df[~((df.name=='Henry Kissinger') & (df.year==1978))]
df = df[~((df.name=='Ragnar Granit') & (df.year==1809))]
df = df[~(df.name=='Marie Curie')]

In [ ]:
df[df.year==1809]

In [ ]:
df.loc[df.name=='Artturi Ilmari Virtanen', 'year'] = 1945
df.loc[df.name=='Frans Eemil Sillanpää', 'year'] = 1939

In [ ]:
df[df.country=='Finland'][["name", "year"]]

In [ ]:
df.loc[(df.name=='Marie Skłodowska-Curie') & (df.year==1911), 'country'] = 'France'

In [ ]:
df[df.duplicated(subset=['name'], keep=False)].sort_values('name')[['name', 'year', 'country']]

## Dealing with missing fields

In [ ]:
df.count()

In [ ]:
df[df.category.isnull()][['name', 'text']]

In [ ]:
df.loc[df.name=='Andrew Schally', 'category'] = 'Physiology or Medicine'
df.loc[df.name=='Róbert Bárány', 'category'] = 'Physiology or Medicine'
df.loc[df.name=='Alexis Carrel', 'category'] = 'Physiology or Medicine'
df.loc[df.name=='Amartya Sen', 'category'] = 'Economics'

In [ ]:
df[df.gender.isnull()]['name']

In [ ]:
df = df[df.gender.notnull()]

In [ ]:
df.count()

## Dealing with times and dates

In [ ]:
with_death_dates = df[df.date_of_death.notnull()]
bad_dates = pd.isnull(pd.to_datetime(with_death_dates.date_of_death, errors='coerce'))
with_death_dates[bad_dates][['name', 'date_of_death']]

In [ ]:
bad_dates = pd.isnull(pd.to_datetime(df.date_of_birth, errors='coerce'))
df[bad_dates][['name', 'date_of_birth']]

In [ ]:
df.date_of_death = pd.to_datetime(df.date_of_death, errors='raise')

In [ ]:
df.date_of_birth = pd.to_datetime(df.date_of_birth, errors='raise')

In [ ]:
df.info()

## Generate the "award age" field

In [ ]:
df['award_age'] = df.year - pd.DatetimeIndex(df.date_of_birth).year

In [ ]:
df.sort_values(['award_age', 'year']).iloc[:15][['name', 'award_age', 'category', 'year']]

## The full function to clean data

In [3]:
def clean_data(df):
    
    df.name = df.name.str.replace('*', '')
    df.name = df.name.str.strip()

    df.replace("", np.nan, inplace=True)

    df_born = df[df.born_in.notnull()][["name", "year", "born_in", "text"]]

    df = df.reindex(np.random.permutation(df.index)).sort_values("born_in")
    df = df.drop_duplicates(['name', 'year'], keep='last')
    df = df.sort_index()

    df = df[~((df.name=='Henry Kissinger') & (df.year==1978))]
    df = df[~((df.name=='Ragnar Granit') & (df.year==1809))]
    df = df[~(df.name=='Marie Curie')]

    df.loc[df.name=='Artturi Ilmari Virtanen', 'year'] = 1945
    df.loc[df.name=='Frans Eemil Sillanpää', 'year'] = 1939

    df.loc[(df.name=='Marie Skłodowska-Curie') & (df.year==1911), 'country'] = 'France'

    df.loc[df.name=='Andrew Schally', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Róbert Bárány', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Alexis Carrel', 'category'] = 'Physiology or Medicine'
    df.loc[df.name=='Amartya Sen', 'category'] = 'Economics'

    df = df[df.gender.notnull()]

    df.date_of_death = pd.to_datetime(df.date_of_death, errors='raise')
    df.date_of_birth = pd.to_datetime(df.date_of_birth, errors='raise')
    df['award_age'] = df.year - pd.DatetimeIndex(df.date_of_birth).year

    return df, df_born

In [4]:
df = load_data('nobel_winners_full.json')
df_clean, df_born = clean_data(df)

In [5]:
df_clean.sort_values(['award_age', 'year']).iloc[:15][['name', 'award_age', 'category', 'year']]

,name,award_age,category,year
670,Malala Yousafzai,17,Peace,2014
419,William Lawrence Bragg,25,Physics,1915
712,Nadia Murad,25,Peace,2018
513,Georges J. F. Köhler,30,Physiology or Medicine,1976
858,Werner Karl Heisenberg,31,Physics,1932
415,Paul Dirac,31,Physics,1933
46,Carl Anderson,31,Physics,1936
99,Tsung-Dao Lee,31,Physics,1957
1040,Frederick G. Banting,32,Physiology or Medicine,1923
894,Rudolf Mössbauer,32,Physics,1961


## Merging dataframes

In [6]:
df_winners_bio = load_data('winners_bio.json')

In [7]:
df_winners_bio.head()

,bio_image,image_urls,link,mini_bio
0,full/9daaecfcf8bb7105837943d03ddcfb665cfd8397.jpg,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/C%C3%A9sar_Milstein,"César Milstein, CH, FRS (8 October 1927 – 24 M..."
1,full/4cb2c33a5dc3682f935daf59d6fcff61eb43b781.jpg,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Fritz_Pregl,
2,full/5ac06410e54227219a7b467108cb0875ee78064b.jpg,[https://upload.wikimedia.org/wikipedia/en/thu...,https://en.wikipedia.org/wiki/Richard_Adolf_Zs...,
3,full/56bd02cfd357fc0e86688eb3c78c0baef4cf10bb.jpg,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Julius_Wagner-Ja...,
4,full/47410cd8d1ce6ce5c6e8c073f5d7347c5db59a22.jpg,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Karl_Landsteiner,"Karl Landsteiner, ForMemRS, (14 June 1868 – 26..."


In [8]:
df_clean.head()

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,award_age
2,NaN,Physics,Austria,1883-06-24,1964-12-17,male,https://en.wikipedia.org/wiki/Victor_Francis_Hess,Victor Francis Hess,Deutschfeistritz,Mount Vernon,"Victor Francis Hess , Physics, 1936",1936,53
3,NaN,Physics,Austria,1887-08-12,1961-01-04,male,https://en.wikipedia.org/wiki/Erwin_Schr%C3%B6...,Erwin Schrödinger,Vienna,Vienna,"Erwin Schrödinger , Physics, 1933",1933,46
4,NaN,Chemistry,Austria,1914-05-19,2002-02-06,male,https://en.wikipedia.org/wiki/Max_Perutz,Max Perutz,Vienna,Cambridge,"Max Perutz , Chemistry, 1962",1962,48
5,NaN,Physics,Austria,1900-04-25,1958-12-15,male,https://en.wikipedia.org/wiki/Wolfgang_Pauli,Wolfgang Pauli,Vienna,Zürich,"Wolfgang Pauli , Physics, 1945",1945,45
6,NaN,Physiology or Medicine,Denmark,1911-12-23,1994-10-07,male,https://en.wikipedia.org/wiki/Niels_Kaj_Jerne,Niels Kaj Jerne,London,Castillon-du-Gard,"Niels Kaj Jerne , Physiology or Medicine, 1984",1984,73


In [9]:
df_winners_all = pd.merge(df_clean, df_winners_bio, how='outer', on='link')

In [10]:
df_winners_all.count()

born_in             33
category          1110
country           1077
date_of_birth     1110
date_of_death      750
gender            1110
link              1124
name              1110
place_of_birth    1110
place_of_death     750
text              1110
year              1110
award_age         1110
bio_image         1096
image_urls        1124
mini_bio          1124
dtype: int64

In [11]:
df_winners_all = df_winners_all[~df_winners_all.name.isnull()].drop_duplicates(['link', 'year'])

In [12]:
df_winners_all.count()

born_in            21
category          920
country           899
date_of_birth     920
date_of_death     625
gender            920
link              920
name              920
place_of_birth    920
place_of_death    625
text              920
year              920
award_age         920
bio_image         899
image_urls        920
mini_bio          920
dtype: int64

In [13]:
df_winners_all[df_winners_all.bio_image.isnull()]

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,award_age,bio_image,image_urls,mini_bio
191,NaN,Physiology or Medicine,United States,1918-01-23,1999-02-21,female,https://en.wikipedia.org/wiki/Gertrude_B._Elion,Gertrude B. Elion,New York City,Chapel Hill,"Gertrude B. Elion , Physiology or Medicine, 1988",1988.0,70.0,NaN,[],"Gertrude Belle Elion (January 23, 1918 – Febru..."
212,NaN,Physiology or Medicine,United States,1903-12-19,1996-06-06,male,https://en.wikipedia.org/wiki/George_Davis_Snell,George D. Snell,Bradford,Bar Harbor,"George D. Snell , Physiology or Medicine, 1980",1980.0,77.0,NaN,[],
217,NaN,Physiology or Medicine,United States,1924-02-23,1998-05-07,male,https://en.wikipedia.org/wiki/Allan_McLeod_Cor...,Allan M. Cormack,Johannesburg,Winchester,"Allan M. Cormack , born in South Africa , Phy...",1979.0,55.0,NaN,[],
256,NaN,Chemistry,United States,1919-04-22,2001-06-17,male,https://en.wikipedia.org/wiki/Donald_J._Cram,Donald J. Cram,Chester,Palm Desert,"Donald J. Cram , Chemistry, 1987",1987.0,68.0,NaN,[],
286,NaN,Economics,United States,1927-08-24,NaT,male,https://en.wikipedia.org/wiki/Harry_M._Markowitz,Harry M. Markowitz,Chicago,NaN,"Harry M. Markowitz , Economics, 1990",1990.0,63.0,NaN,[],"Harry Max Markowitz (born August 24, 1927) is ..."
337,NaN,Economics,United States,1937-09-15,NaT,male,"https://en.wikipedia.org/wiki/Robert_Lucas,_Jr.",Robert Lucas,Yakima,NaN,"Robert Lucas, Jr. , Economics, 1995",1995.0,58.0,NaN,[],
338,NaN,Physiology or Medicine,United States,1947-05-08,NaT,male,https://en.wikipedia.org/wiki/H._Robert_Horvitz,H. Robert Horvitz,Chicago,NaN,"H. Robert Horvitz , Physiology or Medicine, 2002",2002.0,55.0,NaN,[],
340,NaN,Chemistry,United States,1941-04-28,NaT,male,https://en.wikipedia.org/wiki/K._Barry_Sharpless,K. Barry Sharpless,Philadelphia,NaN,"K. Barry Sharpless , Chemistry, 2001",2001.0,60.0,NaN,[],"Karl Barry Sharpless (born April 28, 1941) is ..."
351,NaN,Physiology or Medicine,United States,1939-10-30,NaT,male,https://en.wikipedia.org/wiki/Leland_H._Hartwell,Leland H. Hartwell,Los Angeles,NaN,"Leland H. Hartwell , Physiology or Medicine, 2001",2001.0,62.0,NaN,[],Leland Harrison (Lee) Hartwell (born October 3...
366,NaN,Physics,United States,1949-08-31,NaT,male,https://en.wikipedia.org/wiki/H._David_Politzer,H. David Politzer,New York City,NaN,"H. David Politzer , Physics, 2004",2004.0,55.0,NaN,[],Hugh David Politzer (/ˈpɑːlɪtsər/; born August...


## Saving the cleaned dataset

In [14]:
from tools.mongodb import *

In [15]:
df_to_mongo(df_clean, 'nobel_prize', 'winners')
df_to_mongo(df_born, 'nobel_prize', 'winners_born_in')
df_to_mongo(df_winners_all, 'nobel_prize', 'winners_all')

In [16]:
df_restore = mongo_to_df('nobel_prize', 'winners_all')
df_restore

,award_age,bio_image,born_in,category,country,date_of_birth,date_of_death,gender,image_urls,link,mini_bio,name,place_of_birth,place_of_death,text,year
0,53.0,full/6d9fe4bf7c1ad536496af68bb8df887f8e48fea8.jpg,None,Physics,Austria,1883-06-24T00:00:00.000Z,1964-12-17T00:00:00.000Z,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Victor_Francis_Hess,,Victor Francis Hess,Deutschfeistritz,Mount Vernon,"Victor Francis Hess , Physics, 1936",1936.0
1,46.0,full/939807ca9cad70758e7d1a3abb4157db4b762c98.jpg,None,Physics,Austria,1887-08-12T00:00:00.000Z,1961-01-04T00:00:00.000Z,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Erwin_Schr%C3%B6...,Erwin Rudolf Josef Alexander Schrödinger (UK: ...,Erwin Schrödinger,Vienna,Vienna,"Erwin Schrödinger , Physics, 1933",1933.0
2,48.0,full/8a9f331774b2e3babc81e067167d50aa75aa3c49.jpg,None,Chemistry,Austria,1914-05-19T00:00:00.000Z,2002-02-06T00:00:00.000Z,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Max_Perutz,Max Ferdinand Perutz OM CH CBE FRS (19 May 191...,Max Perutz,Vienna,Cambridge,"Max Perutz , Chemistry, 1962",1962.0
3,45.0,full/25a4916bb7bccaddf1c72df8063313bd36c6c24f.jpg,None,Physics,Austria,1900-04-25T00:00:00.000Z,1958-12-15T00:00:00.000Z,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Wolfgang_Pauli,,Wolfgang Pauli,Vienna,Zürich,"Wolfgang Pauli , Physics, 1945",1945.0
4,73.0,full/30e115ad0db7047e6fb041c9104ee453336f8f1c.jpg,None,Physiology or Medicine,Denmark,1911-12-23T00:00:00.000Z,1994-10-07T00:00:00.000Z,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Niels_Kaj_Jerne,"Niels Kaj Jerne, FRS (23 December 1911 – 7 Oct...",Niels Kaj Jerne,London,Castillon-du-Gard,"Niels Kaj Jerne , Physiology or Medicine, 1984",1984.0
5,73.0,full/14ca8483311f82d0efed2882ae083c27dcd9bc51.jpg,None,Physics,Netherlands,1837-11-23T00:00:00.000Z,1923-03-08T00:00:00.000Z,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Johannes_Diderik...,Johannes Diderik van der Waals (Dutch pronunci...,Johannes Diderik van der Waals,Leiden,Amsterdam,"Johannes Diderik van der Waals , Physics, 1910",1910.0
6,69.0,full/5b5831d284ab0e2d6507134510493cd1b6dcf8e1.jpg,None,Chemistry,Turkey,1946-09-08T00:00:00.000Z,None,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Aziz_Sancar,Sancar is currently the Sarah Graham Kenan Pro...,Aziz Sancar,Savur,None,"Aziz Sancar , Chemistry, 2015",2015.0
7,54.0,full/2c3d6e6298f69c74ffdf0d47d78888dfc4f03d14.jpg,None,Literature,Turkey,1952-06-07T00:00:00.000Z,None,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Orhan_Pamuk,Ferit Orhan Pamuk (generally known simply as O...,Orhan Pamuk,Istanbul,None,"Orhan Pamuk , Literature, 2006",2006.0
8,68.0,full/5657cf32095f6542d6f3dae1419cb0d2c91c37ee.jpg,None,Physics,Ukraine,1924-03-08T00:00:00.000Z,2010-09-29T00:00:00.000Z,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Georges_Charpak,,Georges Charpak,Dubrovytsia,Paris,"Georges Charpak , Physics, 1992",1992.0
9,45.0,full/aa4f9d1b79d90da7384aa324660f9fbd01342d68.jpg,None,Chemistry,United Kingdom,1877-09-01T00:00:00.000Z,1945-11-20T00:00:00.000Z,male,[https://upload.wikimedia.org/wikipedia/common...,https://en.wikipedia.org/wiki/Francis_William_...,Francis William Aston FRS (1 September 1877 – ...,Francis William Aston,Harborne,Cambridge,"Francis William Aston , Chemistry, 1922",1922.0


In [18]:
import sqlalchemy

In [20]:
db = sqlalchemy.create_engine('sqlite:///ch6/nobel_winners/data/nobel_prize.db')
df_clean.to_sql('winners', db)